In [1]:
import torch.nn as nn
import torch

import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
import shutil
import pickle
import random
from tqdm import tqdm

from torchvision.datasets.folder import ImageFolder
from torch.utils.data import DataLoader
from skimage.filters import threshold_local, gaussian

from datetime import datetime


In [2]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/010-CUB-27-imgnet_OOD_cnext26_img=224_nprotos=20'
# run_path = '/home/harishbabu/projects/PIPNet/runs/031-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/032-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/035-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/043-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = "/home/harishbabu/projects/PIPNet/runs/036-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel_uniformity"
# run_path = "/home/harishbabu/projects/PIPNet/runs/041-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/042-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/044-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20-or-4per-desc_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/046-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=10per-desc_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/047-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc"
# run_path = "/home/harishbabu/projects/PIPNet/runs/048-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc_unit-sphere"
# run_path = "/home/harishbabu/projects/PIPNet/runs/051-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/052-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/055-CUB-18_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/056-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/057-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/058-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"

# with unit sphere
# run_path = "/home/harishbabu/projects/PIPNet/runs/059-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# unit sphere with softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/065-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# original hpipnet with 20 protos per node no KO, no OOD, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/062-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD"

# original hpipnet with 20 protos per node no KO, no OOD, WITH tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/063-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD_tanh-desc"

# with unit sphere but no AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/066-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_no-align_no-uni_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc, INCORRECT
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-incorrect-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, no softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/068-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# 071 with bias
# run_path = "/home/harishbabu/projects/PIPNet/runs/071-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_with-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 072 gumbel softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/072-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 073 gumbel softmax, tau-1.0
# run_path = "/home/harishbabu/projects/PIPNet/runs/073-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax-tau=1_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 075 068 with focal loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/075-068-with-focal_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 076 cs followed by softmax. Uses align_pf along with align+uni
# run_path = "/home/harishbabu/projects/PIPNet/runs/076_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_align-pf-during-training_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2-APW=5_batch=20"

# 074 multiply_cs_softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/074-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_multi-cs-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 077 unit sphere protopool with cosin no softmax constant 20 protos per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/077_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20-sphere-protopool_finetune=5_align-pf-during-training_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 082 unit sphere cs followed by softmax with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/082-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 083 unit sphere cs followed by softmax with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/083-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_no-align_no-uni_no-mm-loss_batch=48"

# 085 unit sphere cs followed by softmax-with-tau with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/085-notebook-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=12"

# 091 basic gaussian multiplier on stage 4
# run_path = "/home/harishbabu/projects/PIPNet/runs/091-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=256_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 092 basic gaussian multiplier on stage 3, 4
# run_path = "/home/harishbabu/projects/PIPNet/runs/092-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=3,4|1.0|50_img=224_latent-dim=256_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 093 128 dim linear
# run_path = "/home/harishbabu/projects/PIPNet/runs/093-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=128_nprotos=20_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=1-CW=2_mm-loss_batch=48"

# 094 128 dim linear
# run_path = "/home/harishbabu/projects/PIPNet/runs/094-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=128_nprotos=20_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=1-CW=2_mm-loss_batch=48"

# 095 ablation 091 without AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/095-091-woALUNI-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_no-AL_no-UNI_mm-loss_batch=48"

# 096 ablation 091 without AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/096-091-wfocal-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 097 - 091 with bg
# run_path = "/home/harishbabu/projects/PIPNet/runs/097-091-wbg-CUB-18_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 0100 cub29 with 20 per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/100_CUB-29-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 0101 baseline with 4 per desc per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/101-baseline-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_no-KO_no-OOD"

# 0103 091 with 20 per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/103-091-wProtoPool20PerNode_CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_batch=48"

# 0108 LOU3
run_path = "/home/harishbabu/projects/PIPNet/runs/108-wProtoPool20PerNode_LOU3_CUB-18-imgnet-bg_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_batch=48"

try:
    sys.path.remove('/home/harishbabu/projects/PIPNet')
except:
    pass
sys.path.insert(0, os.path.join(run_path, 'source_clone'))


In [3]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
from util.node import Node
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.func import get_patch_size
from util.data import ModifiedLabelLoader

In [4]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('/home/harishbabu/data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

exp_no = int(os.path.basename(run_path)[:3])

if exp_no < 77:
    if ('num_protos_per_descendant' in args) and (args.num_protos_per_descendant > 0):
        for node in root.nodes_with_children():
            node.set_num_protos(args.num_protos_per_descendant)
else:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))

------------------------- No discretization -------------------------


In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

# args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
# args = pickle.load(args_file)

# ckpt_file_name = 'net_overspecific_pruned_replaced_thresh=0.5_last'
ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)

print(args.batch_size, trainloader.batch_size)

if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
print(net.eval())
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
# with torch.no_grad():
#     xs1, _, _ = next(iter(trainloader))
#     xs1 = xs1.to(device)
#     proto_features, _, _ = net(xs1)
#     wshape = proto_features['root'].shape[-1]
#     args.wshape = wshape #needed for calculating image patch size
#     print("Output shape: ", proto_features['root'].shape, flush=True)
    
args.wshape = 26

/home/harishbabu/.conda/envs/hpnet1/lib/python3.8/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Num classes (k) =  18 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_023_Brandt_Cormorant'] etc.
48 48
Classes:  {'cub_001_Black_footed_Albatross': 0, 'cub_002_Laysan_Albatross': 1, 'cub_003_Sooty_Albatross': 2, 'cub_004_Groove_billed_Ani': 3, 'cub_023_Brandt_Cormorant': 4, 'cub_024_Red_faced_Cormorant': 5, 'cub_025_Pelagic_Cormorant': 6, 'cub_031_Black_billed_Cuckoo': 7, 'cub_032_Mangrove_Cuckoo': 8, 'cub_033_Yellow_billed_Cuckoo': 9, 'cub_045_Northern_Fulmar': 10, 'cub_050_Eared_Grebe': 11, 'cub_051_Horned_Grebe': 12, 'cub_052_Pied_billed_Grebe': 13, 'cub_053_Western_Grebe': 14, 'cub_086_Pacific_Loon': 15, 'cub_100_Brown_Pelican': 16, 'cub_101_White_Pelican': 17}
stage 4
Number of prototypes:  20
----------Prototypes per descendant: 0----------
Assigned 20 protos to node root
Assigned 20 protos to node 052+053
Assigned 20 protos to node 004+086
Assigned 20 protos to node 053+050
Assigned 20 protos to node 00

In [6]:
str(net.module._net).upper().startswith('CONVNEXT')

[i for i in net.module._net.modules() if isinstance(i, nn.Conv2d)][-1].out_channels

768

In [7]:
[i for i in net.module.modules()]

[PIPNet(
   (_net): ConvNeXt(
     (features): Sequential(
       (0): Conv2dNormActivation(
         (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
         (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
       )
       (1): Sequential(
         (0): CNBlock(
           (block): Sequential(
             (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
             (1): Permute()
             (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
             (3): Linear(in_features=96, out_features=384, bias=True)
             (4): GELU(approximate='none')
             (5): Linear(in_features=384, out_features=96, bias=True)
             (6): Permute()
           )
           (stochastic_depth): StochasticDepth(p=0.0, mode=row)
         )
         (1): CNBlock(
           (block): Sequential(
             (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
             (1): Permute()
             (2)

In [8]:
import wandb
from util.log import Log
os.environ['WANDB_DISABLED'] = 'true'
wandb_run = wandb.init(project="pipnet", name=os.path.basename(args.log_dir), config=vars(args), reinit=False)
# log = Log(args.log_dir)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [9]:
optimizer_net, optimizer_classifier, params_to_freeze, params_to_train, params_backbone = get_optimizer_nn(net, args)            
scheduler_net = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, T_max=len(trainloader)*args.epochs, eta_min=args.lr_net/100.)
if args.epochs<=30:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=5, eta_min=0.001, T_mult=1, verbose=False)
else:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=10, eta_min=0.001, T_mult=1, verbose=False)

    
if args.OOD_dataset:
    trainloader_OOD, trainloader_pretraining_OOD, trainloader_normal_OOD, trainloader_normal_augment_OOD, projectloader_OOD, testloader_OOD, test_projectloader_OOD, _ = get_dataloaders(args, device, OOD=True)
    print('-'*25 + 'Using OOD data' + '-'*25)
else:
    trainloader_OOD = trainloader_pretraining_OOD = trainloader_normal_OOD = trainloader_normal_augment_OOD = projectloader_OOD = testloader_OOD = test_projectloader_OOD = None
    print('-'*25 + 'Not using OOD data' + '-'*25)
    
if ('focal_loss' in args) and (args.focal_loss == 'y'):
    from util.custom_losses import WeightedCrossEntropyLoss, WeightedNLLLoss, FocalLossWrapper
    criterion = FocalLossWrapper(device=device, alpha=1, gamma=args.focal_loss_gamma, reduction='mean').to(device)
else:
    from util.custom_losses import WeightedCrossEntropyLoss, WeightedNLLLoss, FocalLossWrapper
    criterion = WeightedNLLLoss(device=device).to(device)

chosen network is convnext
-------------------------Not using OOD data-------------------------


In [8]:
test_info, log_dict = test_pipnet(net, testloader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log)

NameError: name 'log' is not defined

In [13]:
def get_heatmap(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)


def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

def get_bb_gaussian_threshold(latent_activation, sigma=1.0, percentile=97, extend_h=0, extend_w=0):
    # latent_activation -> []
    upscaled_similarity = get_upscaled_activation_uninterpolated(latent_activation, \
                                                                 image_size=(args.image_size, args.image_size))
    upscaled_similarity = minmaxscale(upscaled_similarity)
    upscaled_similarity = gaussian(upscaled_similarity, sigma=sigma)
    upscaled_similarity = threshold_local(upscaled_similarity, block_size=15, method='mean')
    h_min, h_max, w_min, w_max = find_top_percentile_bbox(upscaled_similarity ,percentile=97)
    h_min = max(0, h_min-extend_h)
    h_max = min(upscaled_similarity.shape[0], h_max+extend_h)
    w_min = max(0, w_min-extend_w)
    w_max = min(upscaled_similarity.shape[1], w_max+extend_w)
    return h_min, h_max, w_min, w_max


def minmaxscale(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())

from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import DataLoader

def unshuffle_dataloader(dataloader):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

In [14]:
# Proto activations on leaf descendents - topk images

from util.data import ModifiedLabelLoader
from collections import defaultdict
import heapq
import pdb
from util.vis_pipnet import get_img_coordinates
import torchvision.transforms as transforms
from PIL import ImageFont, Image, ImageDraw as D
import torchvision
import torch.nn.functional as F



# def find_top_percentile_bbox(image, percentile=95):
#     threshold = np.percentile(image.flatten(), percentile)
#     mask = image >= threshold
#     coords = np.argwhere(mask)
#     if coords.size == 0:
#         return None, None, None, None
#     h_min, w_min = coords.min(axis=0)
#     h_max, w_max = coords.max(axis=0)
#     h_min, h_max, w_min, w_max = map(int, [h_min, h_max, w_min, w_max])
#     return h_min, h_max, w_min, w_max

# def find_high_activation_crop(activation_map, percentile=95):
#     threshold = np.percentile(activation_map, percentile)
#     mask = np.ones(activation_map.shape)
#     mask[activation_map < threshold] = 0
#     lower_y, upper_y, lower_x, upper_x = 0, 0, 0, 0
#     for i in range(mask.shape[0]):
#         if np.amax(mask[i]) > 0.5:
#             lower_y = i
#             break
#     for i in reversed(range(mask.shape[0])):
#         if np.amax(mask[i]) > 0.5:
#             upper_y = i
#             break
#     for j in range(mask.shape[1]):
#         if np.amax(mask[:,j]) > 0.5:
#             lower_x = j
#             break
#     for j in reversed(range(mask.shape[1])):
#         if np.amax(mask[:,j]) > 0.5:
#             upper_x = j
#             break
#     return lower_y, upper_y+1, lower_x, upper_x+1

# def get_upscaled_activation_uninterpolated(latent_activation, image_size):
#     image_a = latent_activation.cpu().numpy()
#     min_image_a = image_a.min()
#     max_image_a = image_a.max()
#     image_a = (image_a - min_image_a) / (max_image_a - min_image_a)
#     reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((image_size[-1], \
#                                                                                             image_size[-2]), \
#                                                                                           resample=Image.NEAREST ))
    
#     reshaped_image_a = (reshaped_image_a / 255).astype('float16')
#     reshaped_image_a = (reshaped_image_a * (max_image_a - min_image_a)) + min_image_a
#     return reshaped_image_a

def functional_UnitConv2D(in_features, weight, bias, stride = 1, padding=0):
    normalized_weight = F.normalize(weight.data, p=2, dim=(1, 2, 3)) # Normalize the kernels to unit vectors
    normalized_input = F.normalize(in_features, p=2, dim=1) # Normalize the input to unit vectors
    if bias is not None:
        normalized_bias = F.normalize(bias.data, p=2, dim=0) # Normalize the kernels to unit vectors
    else:
        normalized_bias = None
    return F.conv2d(normalized_input, normalized_weight, normalized_bias, stride=stride, padding=padding)

def findCorrespondingToMax(base, target):
    output, indices = F.max_pool2d(base, kernel_size=(26, 26), return_indices=True)# these are logits
    tensor_flattened = target.view(target.shape[0], target.shape[1], -1)
    indices_flattened = indices.view(target.shape[0], target.shape[1], -1)
    corresponding_values_in_target = torch.gather(tensor_flattened, 2, indices_flattened)
    corresponding_values_in_target = corresponding_values_in_target.view(target.shape[0],\
                                     target.shape[1], 1, 1)
    pooled_target = corresponding_values_in_target
    return pooled_target

def customForwardWithCSandSoftmax(net, xs,  inference=False):
    features = net.module._net(xs) 
    proto_features = {}
    proto_features_cs = {}
    proto_features_softmaxed = {}
    pooled = {}
    pooled_cs = {}
    pooled_softmaxed = {}
    out = {}
    for node in net.module.root.nodes_with_children():
        # this may or may not be cosine similarity based on UniConv2D or Conv2d
        proto_features[node.name] = getattr(net.module, '_'+node.name+'_add_on')(features)
        
        proto_features[node.name] = torch.abs(proto_features[node.name])
        
        #calculating cosine similarity
        prototypes = getattr(net.module, '_'+node.name+'_add_on')
        proto_features_cs[node.name] = functional_UnitConv2D(features, prototypes.weight, prototypes.bias)

        if net.module.args.softmax == 'y':
            softmax_tau = 0.2
            proto_features[node.name] = proto_features[node.name] / softmax_tau
            proto_features_softmaxed[node.name] = net.module._softmax(proto_features[node.name])
            proto_features[node.name] = proto_features_softmaxed[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'
        elif net.module.args.gumbel_softmax == 'y':
            proto_features_softmaxed[node.name] = net.module._gumbel_softmax(proto_features[node.name])
            proto_features[node.name] = proto_features_softmaxed[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'

        if net.module.args.multiply_cs_softmax == 'y':
            proto_features[node.name] = proto_features_cs[node.name] * proto_features_softmaxed[node.name]
        pooled[node.name] = net.module._pool(proto_features[node.name])
        
        # this could be softmax or cosine similarity
        pooled_cs[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_cs[node.name])
        
        pooled_softmaxed[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_softmaxed[node.name])

        if inference:
            pooled[node.name] = torch.where(pooled[node.name] < 0.1, 0., pooled[node.name])  #during inference, ignore all prototypes that have 0.1 similarity or lower
        out[node.name] = getattr(net.module, '_'+node.name+'_classification')(pooled[node.name]) #shape (bs*2, num_classes) # these are logits

    return features, proto_features, pooled, pooled_cs, pooled_softmaxed, out

find_non_descendants = False # True, False # param
vizloader_name = 'projectloader'
bbox_percentile = 97
topk = 10 # param, args param
save_images = True #True
pruning_threshold = 0.4

font = ImageFont.truetype("arial.ttf", 50)
font2 = ImageFont.truetype("arial.ttf", 20)

from datetime import datetime
# txt_file = open(os.path.join(run_path, "num_proto_details_"+datetime.now().strftime("%m:%d:%H:%M:%S")+".txt"), "a")
# txt_file.write('\n')

def write_num_proto_details(proto_mean_activations, node_name, net, threshold, txt_file, args):
    
    rand_input = torch.randn((1, 3, args.image_size, args.image_size))
    with torch.no_grad():
        *_, pooled, out = net(rand_input)
    num_protos = pooled[node_name].shape[1]
    used_protos = len(proto_mean_activations)
    non_overspecific = 0
    for p in proto_mean_activations:
        logstr = '\t'*2 + f'Proto:{p} '
        protos_mean_for_all_leaf_descedants = []
        for leaf_descendent in proto_mean_activations[p]:
            mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
            protos_mean_for_all_leaf_descedants.append(mean_activation)
            
        if all([(mean_activation>0.2) for mean_activation in protos_mean_for_all_leaf_descedants]):
            non_overspecific += 1
            
    txt_file.write(f"Node:{node_name},Total:{num_protos},Used:{used_protos},Good:{non_overspecific},threshold={threshold}\n")


def get_heap():
    list_ = []
    heapq.heapify(list_)
    return list_

patchsize, skip = get_patch_size(args)

vizloader_dict = {'trainloader': trainloader,
                 'projectloader': projectloader,
                 'testloader': testloader,
                 'test_projectloader': test_projectloader}
# vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])


if type(vizloader_dict[vizloader_name].dataset) == ImageFolder:
    name2label = vizloader_dict[vizloader_name].dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
else:
    name2label = vizloader_dict[vizloader_name].dataset.dataset.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}

for node in root.nodes_with_children():
#     if node.name == 'root':
#         continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

#     name2label = projectloader.dataset.class_to_idx # param
#     label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(vizloader_dict[vizloader_name], node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}
    
    imgs = modifiedLabelLoader.filtered_imgs

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> list of top-k activations
    proto_mean_activations = defaultdict(lambda: defaultdict(get_heap))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
        # change
#         if not find_non_descendants: 
#             # do only when finding descendants
#             if coarse_label2name[ys.item()] not in non_leaf_children_names:
#                 continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            
            model_output = net(xs, inference=False)
            if len(model_output) == 3:
                softmaxes, pooled, _ = model_output
            elif len(model_output) == 4:
                _, softmaxes, pooled, _ = model_output
            
            _, _, _, pooled_ip, _, _ = customForwardWithCSandSoftmax(net, xs, inference=False)
            
            pooled = pooled[node.name].squeeze(0)
            pooled_ip = pooled_ip[node.name].squeeze(0) 
            softmaxes = softmaxes[node.name]#.squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]
                
                # Take the max per prototype.                             
                max_per_prototype, max_idx_per_prototype = torch.max(softmaxes, dim=0)
                max_per_prototype_h, max_idx_per_prototype_h = torch.max(max_per_prototype, dim=1)
                max_per_prototype_w, max_idx_per_prototype_w = torch.max(max_per_prototype_h, dim=1) #shape (num_prototypes)
                
                h_idx = max_idx_per_prototype_h[p, max_idx_per_prototype_w[p]]
                w_idx = max_idx_per_prototype_w[p]

                if len(relevant_proto_class_names) == 0:
                    continue
                
                # change
#                 if (len(relevant_proto_class_names) == 1):# and (relevant_proto_class_names[0] not in non_leaf_children_names):
#                     continue
                
                h_coor_min, h_coor_max, w_coor_min, w_coor_max = get_img_coordinates(args.image_size, softmaxes.shape, patchsize, skip, h_idx, w_idx)
                latent_activation = softmaxes[:, p, :, :]
                if not find_non_descendants:
                    if (coarse_label2name[ys.item()] in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7]
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), pooled_ip[p].item(), xs,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), \
                                               latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), pooled_ip[p].item(), xs,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), \
                                            latent_activation))
                else:
                    if (coarse_label2name[ys.item()] not in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), pooled_ip[p].item(), xs,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), \
                                               latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), pooled_ip[p].item(), xs,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), \
                                            latent_activation))
                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)
    
    with torch.no_grad():
#         class_and_prototypes_post_pruning = copy.deepcopy(class_and_prototypes)
        classification_weights_pre_pruning = classification_weights.clone()
        for p in proto_mean_activations:
            mean_acts_for_all_leafs = []
            for leaf_descendant in proto_mean_activations[p]:
                mean_activation = np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendant]])
                mean_acts_for_all_leafs.append(mean_activation)
            if (np.array(mean_acts_for_all_leafs) < pruning_threshold).any():
                # setting all weights to zero
                classification_weights[:, p] = 0.
#             if node.name == '004+086' and p == 8:
#                 pdb.set_trace()
        for child_classname in class_and_prototypes:
            child_label = node.children_to_labels[child_classname]
            print('Node:', node.name, 'Child:', child_classname, \
                  'Num protos pre pruning:', torch.nonzero(classification_weights_pre_pruning[child_label, :] > 1e-3).shape[0], \
                    'Post pruning:', torch.nonzero(classification_weights[child_label, :] > 1e-3).shape[0])
    
print('Done !!!')

Node: root Child: 052+053 Num protos pre pruning: 5 Post pruning: 1
Node: root Child: 004+086 Num protos pre pruning: 3 Post pruning: 2


Node: 052+053 Child: cub_052_Pied_billed_Grebe Num protos pre pruning: 4 Post pruning: 3
Node: 052+053 Child: 053+050 Num protos pre pruning: 2 Post pruning: 2


Node: 004+086 Child: 004+032 Num protos pre pruning: 1 Post pruning: 1
Node: 004+086 Child: 086+045 Num protos pre pruning: 3 Post pruning: 2


Node: 053+050 Child: cub_053_Western_Grebe Num protos pre pruning: 6 Post pruning: 4
Node: 053+050 Child: 050+051 Num protos pre pruning: 3 Post pruning: 2


Node: 004+032 Child: cub_004_Groove_billed_Ani Num protos pre pruning: 2 Post pruning: 2
Node: 004+032 Child: 032+033 Num protos pre pruning: 2 Post pruning: 1


...

Node: 045+101 Child: 045+003 Num protos pre pruning: 1 Post pruning: 1
Node: 045+101 Child: 101+023 Num protos pre pruning: 1 Post pruning: 1


Node: 045+003 Child: 003+002 Num protos pre pruning: 2 Post pruning: 2
Node: 045+003 Child: cub_045_Northern_Fulmar Num protos pre pruning: 16 Post pruning: 1


Node: 101+023 Child: 101+100 Num protos pre pruning: 1 Post pruning: 1
Node: 101+023 Child: 023+025 Num protos pre pruning: 2 Post pruning: 2


Node: 003+002 Child: cub_003_Sooty_Albatross Num protos pre pruning: 15 Post pruning: 1
Node: 003+002 Child: 002+001 Num protos pre pruning: 2 Post pruning: 2


Node: 023+025 Child: cub_023_Brandt_Cormorant Num protos pre pruning: 16 Post pruning: 2
Node: 023+025 Child: 025+024 Num protos pre pruning: 3 Post pruning: 3
Done !!!


In [16]:
log = Log(os.path.join(args.log_dir, f'post_pruning_thresh={pruning_threshold}'))

In [15]:
test_info, log_dict = test_pipnet(net, testloader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log)

Test Epoch0: 100% 11/11 [00:36<00:00,  3.29s/it, L:-5.601,LC:0.171, LA:0.00, L_UNI:-3.806, LT:-5.000, L_MM:-5.000, L_OOD:-5.000, L_ORTH:-5.000, losses_used:AL+UNI+CL]

	Fine accuracy: 0.79
	Node name: root, acc: 96.48, f1:96.37, samples: 1022, 052+053=204/240=0.85, 004+086=782/782=1.0
	Node name: 052+053, acc: 100.0, f1:100.0, samples: 240, cub_052_Pied_billed_Grebe=60/60=1.0, 053+050=180/180=1.0
	Node name: 004+086, acc: 96.68, f1:96.63, samples: 782, 004+032=202/224=0.9, 086+045=554/558=0.99
	Node name: 053+050, acc: 97.78, f1:97.76, samples: 180, cub_053_Western_Grebe=56/60=0.93, 050+051=120/120=1.0
	Node name: 004+032, acc: 97.32, f1:97.36, samples: 224, cub_004_Groove_billed_Ani=60/60=1.0, 032+033=158/164=0.96
	Node name: 086+045, acc: 89.25, f1:84.18, samples: 558, cub_086_Pacific_Loon=0/60=0.0, 045+101=498/498=1.0
	Node name: 050+051, acc: 80.0, f1:79.98, samples: 120, cub_050_Eared_Grebe=50/60=0.83, cub_051_Horned_Grebe=46/60=0.77
	Node name: 032+033, acc: 95.12, f1:94.97, samples: 164, cub_032_Mangrove_Cuckoo=38/46=0.83, 033+031=118/118=1.0
	Node name: 045+101, acc: 97.59, f1:97.59, samples: 498, 045+003=230/236=0.97, 101+023=256/262=0.98
	N

In [17]:
torch.save({'model_state_dict': net.state_dict(), \
            'optimizer_net_state_dict': optimizer_net.state_dict(), \
            'optimizer_classifier_state_dict': optimizer_classifier.state_dict()}, \
           os.path.join(os.path.join(log._log_dir, 'checkpoints'), 'net_trained_last'))

In [22]:
from torch.utils.data import Sampler, SubsetRandomSampler

def create_filtered_dataloader(dataloader, new_sampler):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        sampler=new_sampler,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

leave_out_loader = testloader
# leave_out_loader = test_projectloader
unique_labels = set()
total_samples = 0
for xs, ys in testloader:
    unique_labels.update(ys.tolist())
    total_samples += xs.shape[0]
print("testloader Unique Labels:", unique_labels)
print("testloader total_samples:", total_samples)

if ('leave_out_classes' in args) and (args.leave_out_classes != ''):
    leave_out_classes = args.leave_out_classes.split(',')[2]
    print('leave_out_classes', leave_out_classes)
    idx_of_classes_to_keep = set()
    name2label = leave_out_loader.dataset.class_to_idx # param
    label2name = {label:name for name, label in name2label.items()}
    for label in label2name:
        # NOTE: Keeping the left out classes here
        if label2name[label] in leave_out_classes:
            idx_of_classes_to_keep.add(label)

    target_indices = []
    for i in range(len(leave_out_loader.dataset)):
        *_, label = leave_out_loader.dataset[i]
        if label in idx_of_classes_to_keep:
            target_indices.append(i)
    sampler = SubsetRandomSampler(target_indices)
    to_shuffle = False
else:
    print('Model not trained with LOU')
    
leave_out_loader = create_filtered_dataloader(leave_out_loader, sampler)
unique_labels = set()
total_samples = 0
for xs, ys in leave_out_loader:
    unique_labels.update(ys.tolist())
    total_samples += xs.shape[0]
print("leave_out_loader Unique Labels:", unique_labels)
print("leave_out_loader total_samples:", total_samples)

name2label = leave_out_loader.dataset.class_to_idx
label2name = {label:name for name, label in name2label.items()}

print('Leave out classes', args.leave_out_classes)

testloader Unique Labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
testloader total_samples: 1022
leave_out_classes cub_032_Mangrove_Cuckoo
leave_out_loader Unique Labels: {8}
leave_out_loader total_samples: 46
Leave out classes cub_003_Sooty_Albatross,cub_051_Horned_Grebe,cub_032_Mangrove_Cuckoo


In [23]:
leave_out_loader.dataset.class_to_idx

{'cub_001_Black_footed_Albatross': 0,
 'cub_002_Laysan_Albatross': 1,
 'cub_003_Sooty_Albatross': 2,
 'cub_004_Groove_billed_Ani': 3,
 'cub_023_Brandt_Cormorant': 4,
 'cub_024_Red_faced_Cormorant': 5,
 'cub_025_Pelagic_Cormorant': 6,
 'cub_031_Black_billed_Cuckoo': 7,
 'cub_032_Mangrove_Cuckoo': 8,
 'cub_033_Yellow_billed_Cuckoo': 9,
 'cub_045_Northern_Fulmar': 10,
 'cub_050_Eared_Grebe': 11,
 'cub_051_Horned_Grebe': 12,
 'cub_052_Pied_billed_Grebe': 13,
 'cub_053_Western_Grebe': 14,
 'cub_086_Pacific_Loon': 15,
 'cub_100_Brown_Pelican': 16,
 'cub_101_White_Pelican': 17}

In [24]:
# testloader

log = Log(os.path.join(args.log_dir, f'LOU_accuracy'))
test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log)

Test Epoch0: 100% 1/1 [00:03<00:00,  3.77s/it, L:1.701,LC:1.603, LA:0.00, L_UNI:-3.708, LT:-5.000, L_MM:-5.000, L_OOD:-5.000, L_ORTH:-5.000, losses_used:AL+UNI+CL]

	Fine accuracy: 0.0
	Node name: root, acc: 100.0, f1:100.0, samples: 92, 052+053=0/0=nan, 004+086=92/92=1.0
	Node name: 052+053, acc: inf, f1:inf, samples: 0, cub_052_Pied_billed_Grebe=0/0=nan, 053+050=0/0=nan
	Node name: 004+086, acc: 86.96, f1:93.02, samples: 92, 004+032=80/92=0.87, 086+045=0/0=nan
	Node name: 053+050, acc: inf, f1:inf, samples: 0, cub_053_Western_Grebe=0/0=nan, 050+051=0/0=nan
	Node name: 004+032, acc: 91.3, f1:95.45, samples: 92, cub_004_Groove_billed_Ani=0/0=nan, 032+033=84/92=0.91
	Node name: 086+045, acc: inf, f1:inf, samples: 0, cub_086_Pacific_Loon=0/0=nan, 045+101=0/0=nan
	Node name: 050+051, acc: inf, f1:inf, samples: 0, cub_050_Eared_Grebe=0/0=nan, cub_051_Horned_Grebe=0/0=nan
	Node name: 032+033, acc: 0.0, f1:0.0, samples: 92, cub_032_Mangrove_Cuckoo=0/92=0.0, 033+031=0/0=nan
	Node name: 045+101, acc: inf, f1:inf, samples: 0, 045+003=0/0=nan, 101+023=0/0=nan
	Node name: 033+031, acc: inf, f1:inf, samples: 0, cub_033_Yellow_billed_Cuckoo=0/0=nan, cub_031_Bl

In [12]:
# test_projectloader

log = Log(os.path.join(args.log_dir, f'LOU_accuracy'))
test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log)

Test Epoch0: 100% 162/162 [00:05<00:00, 31.72it/s, L:2.770,LC:1.627, LA:0.00, L_UNI:-3.416, LT:-5.000, L_MM:-5.000, L_OOD:-5.000, L_ORTH:-5.000, losses_used:AL+UNI+CL]
/home/harishbabu/.conda/envs/hpnet1/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/harishbabu/.conda/envs/hpnet1/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


	Fine accuracy: 0.0
	Node name: root, acc: 85.8, f1:84.96, samples: 324, 052+053=76/120=0.63, 004+086=202/204=0.99
	Node name: 052+053, acc: 95.0, f1:97.44, samples: 120, cub_052_Pied_billed_Grebe=0/0=nan, 053+050=114/120=0.95
	Node name: 004+086, acc: 88.24, f1:88.17, samples: 204, 004+032=76/92=0.83, 086+045=104/112=0.93
	Node name: 053+050, acc: 91.67, f1:95.65, samples: 120, cub_053_Western_Grebe=0/0=nan, 050+051=110/120=0.92
	Node name: 004+032, acc: 86.96, f1:93.02, samples: 92, cub_004_Groove_billed_Ani=0/0=nan, 032+033=80/92=0.87
	Node name: 086+045, acc: 100.0, f1:100.0, samples: 112, cub_086_Pacific_Loon=0/0=nan, 045+101=112/112=1.0
	Node name: 050+051, acc: 0.0, f1:0.0, samples: 120, cub_050_Eared_Grebe=0/0=nan, cub_051_Horned_Grebe=0/120=0.0
	Node name: 032+033, acc: 0.0, f1:0.0, samples: 92, cub_032_Mangrove_Cuckoo=0/92=0.0, 033+031=0/0=nan
	Node name: 045+101, acc: 89.29, f1:94.34, samples: 112, 045+003=100/112=0.89, 101+023=0/0=nan
	Node name: 033+031, acc: inf, f1:inf, 

In [12]:
import torch

type(torch.divide(1, 0).item())

float

In [16]:
os.path.exists('/fastscratch/harishbabu/data/CUB_18_hpnet/dataset_imgnet_hpnet_bb_crop/train_augmented/')

False